In [1]:
!pip install AugmentedSocialScientist pandas numpy

In [2]:
import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'Tesla T4'

In [3]:
import pandas as pd

annotations_file = pd.read_csv("../data/intermediate/current_annotation_file.csv")
annotations_file.sample(5)

,sentence_id,Personne,sentence_text
269,1060206,Personne à protéger,"10 Conférence de J.P. Moatti, Journées nation..."
1270,1390670,NaN,"Enfin, il remplit un formulaire pour choisir l..."
1373,490243,NaN,"Reste que partout dans le monde, l'introductio..."
1032,820142,NaN,"C’est un ensemble de traits \nqui bougent, sou..."
1438,820333,NaN,a. Le problème du traitement immunodépresseur ...


In [4]:
annotations = annotations_file.dropna()
annotations.sample(5)

,sentence_id,Personne,sentence_text
425,930493,Genre/espèce humaine,De plus il importe ici de se souvenir de ce qu...
243,1200618,Genre/espèce humaine,"Dans\t\n ce\t\n contexte,\t\n ne\t\n pourr..."
629,1280759,Individu,Force est de constater que les logiques actuel...
208,940717,Personne à protéger,3. Lors du jugement: \n \n- Faire en sorte qu...
389,1190328,Personne relationnelle,Cette information doit \nêtre adaptée par sa c...


In [5]:
annotations["Personne"].value_counts()

Personne
Personne à protéger       143
Matériel biologique       127
Individu                  118
Agent responsable         104
Personne relationnelle     97
Genre/espèce humaine       90
Humain comme empathie      10
Poubelle                    8
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.90)

In [7]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: Tesla T4


In [8]:
batch_size = 16
train_loader = bert.encode(train_data.sentence_text.values,
train_data.Personne.values, batch_size=batch_size)

  0%|          | 0/627 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (620 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/627 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [9]:
test_loader = bert.encode(test_data.sentence_text.values,
test_data.Personne.values, batch_size=batch_size)

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6}


In [10]:
scores = bert.run_training(
    train_loader,
    test_loader,
    n_epochs=2,
    lr=2e-5,
    random_state=1968,
    save_model_as='clickbait'
    )

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/mamba/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 2 ========
Training...


../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import numpy as np
pred_data = annotations_file[annotations_file.isna().any(axis=1)].drop('Personne', axis=1).dropna()
pred_loader = bert.encode(pred_data.sentence_text.values, batch_size=batch_size)
pred_proba = bert.predict_with_model(pred_loader, model_path='./models/clickbait')
pred_data['pred_label'] = np.argmax(pred_proba, axis=1)
pred_data['pred_proba'] = np.max(pred_proba, axis=1)

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}
